Import necessary dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F
import torch
from torch import optim
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from tqdm import tqdm
from datetime import datetime

In [2]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# elif torch.backends.mps.is_available():
#     device = torch.device("mps:0")
else:
    device = torch.device("cpu")

Load data

In [4]:
data = pd.read_csv('IoT_Modbus.csv')

# TODO: Complete EDA

Combine 'date' and 'time' into a single datetime column

In [5]:
data['date'] = data['date'].str.strip()
data['time'] = data['time'].str.strip()

In [6]:
data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'], format='%d-%b-%y %H:%M:%S')

Extract time features

In [7]:
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day
data['hour'] = data['datetime'].dt.hour
data['minute'] = data['datetime'].dt.minute
data['second'] = data['datetime'].dt.second
data['dayofweek'] = data['datetime'].dt.dayofweek

Time series models need to ensure that the data set is arranged in time order

In [8]:
# Sort the data by datetime
data = data.sort_values(by='datetime')

# Drop the original date, time, and timestamp columns
data.drop(['date', 'time', 'datetime', 'type'], axis=1, inplace=True)

# Adjust feature order
order = ['year', 'month', 'day', 'hour', 'minute', 'second', 'dayofweek', 'FC1_Read_Input_Register', 'FC2_Read_Discrete_Value', 'FC3_Read_Holding_Register', 'FC4_Read_Coil', 'label']
data = data[order].astype('int32')

# Split the dataset (Sequential Split)

In [9]:
# Calculate split points
split_idx = int(len(data) * 0.8)

# Split the data set, keeping order
train_data = data.iloc[:split_idx]
test_data = data.iloc[split_idx:]

# Separate features and labels
X_train = train_data.drop('label', axis=1)
y_train = train_data['label']
X_test = test_data.drop('label', axis=1)
y_test = test_data['label']

In [10]:
y_test.value_counts()

label
0    49527
1     7912
Name: count, dtype: int64

# Data preprocessing (Normalization)

In [11]:
feature_columns = X_train.columns
scaler = MinMaxScaler()

X_train[feature_columns] = scaler.fit_transform(X_train[feature_columns]).astype('float32')
X_test[feature_columns] = scaler.transform(X_test[feature_columns]).astype('float32')
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229755 entries, 541 to 184368
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   year                       229755 non-null  float32
 1   month                      229755 non-null  float32
 2   day                        229755 non-null  float32
 3   hour                       229755 non-null  float32
 4   minute                     229755 non-null  float32
 5   second                     229755 non-null  float32
 6   dayofweek                  229755 non-null  float32
 7   FC1_Read_Input_Register    229755 non-null  float32
 8   FC2_Read_Discrete_Value    229755 non-null  float32
 9   FC3_Read_Holding_Register  229755 non-null  float32
 10  FC4_Read_Coil              229755 non-null  float32
dtypes: float32(11)
memory usage: 11.4 MB


# Execution model
## Create model

In [12]:
class LightweightLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, width_multiplier=1.0):
        super(LightweightLSTM, self).__init__()
        # Adjust hidden size based on the width multiplier
        adjusted_hidden_size = int(hidden_size * width_multiplier)

        # Define the LSTM layer
        self.lstm = nn.LSTM(input_size, adjusted_hidden_size, num_layers=num_layers, batch_first=True)

        self.linear_1 = nn.Linear(adjusted_hidden_size, hidden_size)
        self.linear_2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # LSTM layer
        lstm_out, _ = self.lstm(x)

        # Take the output of the last time step
        last_time_step_out = lstm_out[:, -1, :]

        # Output layer
        x = F.relu(self.linear_1(last_time_step_out))
        out = self.linear_2(x)
        return out

Initialize model

In [13]:
features_num = X_train.shape[1]
hidden_neurons_num = 512
output_neurons_num = 1
lstm_num_layers = 2
multiplier = 0.5

model = LightweightLSTM(features_num, hidden_neurons_num, output_neurons_num, lstm_num_layers, multiplier).to(device)

Build loss functions and optimizers

In [14]:
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.005)

Construct Data Loader

In [15]:
batch_size = 128

# Use to_numpy is much better than .values
X_train_tensor: torch.Tensor = torch.tensor(X_train.to_numpy()).float().unsqueeze(1).to(device)
X_test_tensor: torch.Tensor = torch.tensor(X_test.to_numpy()).float().unsqueeze(1).to(device)

y_train_tensor: torch.Tensor = torch.tensor(y_train.to_numpy()).float().unsqueeze(1).to(device)
y_test_tensor: torch.Tensor = torch.tensor(y_test.to_numpy()).float().unsqueeze(1).to(device)


train_dataset: torch.utils.data.dataset.TensorDataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader: torch.utils.data.dataloader.DataLoader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

test_dataset: torch.utils.data.dataset.TensorDataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader: torch.utils.data.dataloader.DataLoader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Training model

In [16]:
num_epochs = 100
pbar = tqdm(total=num_epochs)
loss_list = [None] * num_epochs
acc_list = [None] * num_epochs
loss_list_train = [None] * num_epochs


for epoch in range(num_epochs):
    model.train()
    loss_storage = 0
    for inputs, labels in train_loader:    
        # FP
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss_storage += loss.item() * inputs.size(0)

        # BP and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate indicators
    model.eval()

    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            
            probabilities = torch.sigmoid(outputs)
            
            loss = criterion(probabilities, labels)
            
            # Calculate indicators
            running_loss += loss.item() * inputs.size(0)

            predictions = (probabilities > 0.5).float().to(device)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    epoch_train_loss = loss_storage / len(train_loader.dataset)
    epoch_val_loss = running_loss / len(test_loader.dataset)
    accuracy = accuracy_score(all_labels, all_preds)
    
    loss_list[epoch] = epoch_val_loss
    acc_list[epoch] = accuracy
    
    pbar.update(1)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss_train: {epoch_train_loss}, Loss_test: {epoch_val_loss}, Accuracy_test: {accuracy}')

pbar.close()

  1%|          | 1/100 [00:12<21:25, 12.98s/it]

Epoch [1/100], Loss_train: 0.6365999746622278, Loss_test: 1.1085409462876354, Accuracy_test: 0.1377461306777625


  2%|▏         | 2/100 [00:25<21:04, 12.91s/it]

Epoch [2/100], Loss_train: 0.38567006738586024, Loss_test: 1.0704310807712887, Accuracy_test: 0.1377461306777625


  3%|▎         | 3/100 [00:38<20:45, 12.84s/it]

...

 22%|██▏       | 22/100 [04:45<16:43, 12.86s/it]

Epoch [22/100], Loss_train: 0.5197935426088104, Loss_test: 0.9545356263693321, Accuracy_test: 0.1377461306777625


 23%|██▎       | 23/100 [04:58<16:27, 12.82s/it]

Epoch [23/100], Loss_train: 0.5197935083307241, Loss_test: 0.9545353402060203, Accuracy_test: 0.1377461306777625


KeyboardInterrupt: 

Visualizing the training process

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(loss_list, label='Training Loss')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Draw accuracy curve
plt.subplot(1, 2, 2)
plt.plot(acc_list, label='Training Accuracy')
plt.title('Training Accuracy per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Save model

In [ ]:
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
model_filename = f"model_{current_time}.pt"
torch.save(model.state_dict(), model_filename)

print("Model saved as:", model_filename)